# DEEP LEARNING

In [12]:
import torch

In [13]:
torch.__version__

'2.9.1+cu130'

In [14]:
from PIL import Image
import numpy as np

In [15]:
# setting a random seed and checking if cuda is available
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [16]:
import os
from pathlib import Path

In [17]:
# checking a random image
img = Image.open(Path('data/train/curly/00cbad1ffe22d900018e5a2e7376daed4.jpg')).resize((200,200))

X = np.array(img )
X.shape

(200, 200, 3)

# DATA PREPARATION

In [18]:
# image dataset prep
from torch.utils.data import Dataset
from torchvision import transforms

In [19]:
# image preparation
class HairDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
		
		    # FOR EACH LABEL PATH
            for img_name in os.listdir(label_dir):

			    # APPEND THE PATHS OF EACH IMAGE
                self.image_paths.append(os.path.join(label_dir, img_name))

			    # APPENDING THE MAPPED INDICES AND LABELS
                self.labels.append(self.class_to_idx[label_name])

	# METHOD TO GET THE LENGTH OF EACH IMAGE PATH
    def __len__(self):
        return len(self.image_paths)

	# METHOD FOR TRANSFORMATION OF EACH IMAGE
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]


	    # BY DEFAULT NO TRANSFORMATION OCCURS AS IT IS SET TO None
        if self.transform:
            image = self.transform(image)
	
        # RETURNS THE TRANSFORMED IMAGE AND LABEL
        return image, label

In [20]:
# input image size
input_size = 200

# normalization values for rgb values after totensor transformation
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# image transformations for training and test set
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

"""
# image transformations for training and test set with data augmentation
train_transforms = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(input_size, scale=(0.9,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

test_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])
"""

'\n# image transformations for training and test set with data augmentation\ntrain_transforms = transforms.Compose([\n    transforms.RandomRotation(10),\n    transforms.RandomResizedCrop(input_size, scale=(0.9,1.0)),\n    transforms.RandomHorizontalFlip(),\n    transforms.ToTensor(),\n    transforms.Normalize(mean=mean, std=std)\n])\n\ntest_transforms = transforms.Compose([\n    transforms.Resize((input_size, input_size)),\n    transforms.ToTensor(),\n    transforms.Normalize(mean=mean, std=std)\n])\n'

In [21]:
from torch.utils.data import DataLoader

In [22]:
# loading the images
train_dataset = HairDataset(
    data_dir='./data/train',
    transform=train_transforms
)

test_dataset = HairDataset(
    data_dir='./data/test',
    transform=test_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

In [23]:
# checking the train loader
ips, labels = next(iter(train_loader))

print(ips[0])
print()
print(labels)

tensor([[[ 0.7077,  0.7419,  0.9646,  ...,  1.3584,  1.3413,  1.3413],
         [ 0.2624,  0.5364,  0.9474,  ...,  1.3584,  1.3584,  1.3413],
         [ 0.0398,  0.2967,  0.9132,  ...,  1.3584,  1.3584,  1.3755],
         ...,
         [-1.4329, -1.3987, -1.3987,  ..., -1.4329, -1.4329, -1.4158],
         [-1.4158, -1.4158, -1.4158,  ..., -1.4329, -1.4329, -1.4329],
         [-1.4158, -1.3987, -1.4158,  ..., -1.4329, -1.4329, -1.4158]],

        [[ 0.1001,  0.1877,  0.4503,  ...,  0.7654,  0.7479,  0.7304],
         [-0.2850, -0.0049,  0.4503,  ...,  0.7654,  0.7654,  0.7304],
         [-0.4251, -0.1975,  0.4153,  ...,  0.7654,  0.7654,  0.7654],
         ...,
         [-1.4930, -1.4580, -1.4580,  ..., -1.4580, -1.4580, -1.4580],
         [-1.4755, -1.4755, -1.4755,  ..., -1.4405, -1.4580, -1.4580],
         [-1.4755, -1.4580, -1.4755,  ..., -1.4580, -1.4580, -1.4405]],

        [[ 0.1128,  0.2173,  0.4962,  ...,  0.6705,  0.6356,  0.6182],
         [-0.1835,  0.0953,  0.5311,  ...,  0